In [1]:
def f(x):
    return x - 1

def g(x):
    return 2 * x

def h(x, y):
    return int(str(x) + str(y))

In [2]:
from collections import deque

def find_shortest_expression(target=2024):
    queue = deque([(5, "5")])
    visited = {5: "5"}
    found_nums = {5: "5"}
    
    print(f"开始搜索目标: {target} ...")
    
    steps = 0
    while queue:
        val, expr = queue.popleft()
        steps += 1

        if val == target:
            print(f"\n成功找到目标!")
            print(f"总计算步数: {steps}")
            return expr

        if steps % 2000 == 0:
            print(f"正在搜索... 已处理 {steps} 个节点, 当前队列长度: {len(queue)}, 当前正在处理数字: {val}")

        moves = []
        
        if val > 1:
            moves.append((f(val), f"f({expr})"))
        
        if val < target * 2:
            moves.append((g(val), f"g({expr})"))
        
        for other_val, other_expr in list(found_nums.items()):
            res_a = h(val, other_val)
            if res_a <= target * 2:
                moves.append((res_a, f"h({expr},{other_expr})"))
            
            res_b = h(other_val, val)
            if res_b <= target * 2:
                moves.append((res_b, f"h({other_expr},{expr})"))

        for n_val, n_expr in moves:
            if n_val not in visited:
                visited[n_val] = n_expr
                found_nums[n_val] = n_expr
                queue.append((n_val, n_expr))
                
            elif len(n_expr) < len(visited[n_val]):
                visited[n_val] = n_expr
                found_nums[n_val] = n_expr
                queue.append((n_val, n_expr))

In [3]:
result = find_shortest_expression(2024)

print("-" * 30)
print(f"数字 2024 的最短表达式是:\n{result}")
print(f"字符长度: {len(result)}")
print("-" * 30)

print(f"验算结果: {eval(result)}")

开始搜索目标: 2024 ...
正在搜索... 已处理 2000 个节点, 当前队列长度: 4167, 当前正在处理数字: 888

成功找到目标!
总计算步数: 2592
------------------------------
数字 2024 的最短表达式是:
h(h(g(g(5)),f(f(f(5)))),f(5))
字符长度: 29
------------------------------
验算结果: 2024


In [ ]:
# ----------------------
# 1. 基础函数定义
# ----------------------
def f(x): return x - 1
def g(x): return 2 * x
def h(x, y): return int(str(x) + str(y))

# ----------------------
# 2. 类的定义 (题目原版)
# ----------------------
class Number: # 表达式树的叶子节点
    """表示一个基础数字，比如 5"""
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return str(self.value)

class Call: # 表达式树的内部节点，表示函数调用 f(5)=Call(f, [Number(5)])
    """表示一个函数调用，比如 f(5) 或 h(5, 5)"""
    def __init__(self, f, operands):
        self.f = f
        self.operands = operands # List of Number or Call objects
        self.value = f(*[e.value for e in operands])

    def __str__(self):
        # 如 "h(5, 5)"
        return f'{self.f.__name__}({", ".join(map(str, self.operands))})'

# ----------------------
# 3. 核心生成器 (题目原版)
# ----------------------
def smalls(n):
    """生成所有复杂度为 n 的表达式对象"""
    if n == 0:
        yield Number(5)  # 基础情况：只有数字 5
    else:
        # 1. 尝试套一层 f 或 g
        for operand in smalls(n-1):
            yield Call(f, [operand])
            yield Call(g, [operand])
            
        # 2. 尝试 h 拼接
        for k in range(n):
            for first in smalls(k):
                for second in smalls(n-k-1):
                    # 剪枝：避免拼接负数
                    if first.value > 0 and second.value > 0:
                        yield Call(h, [first, second])

# ----------------------
# 4. 搜索脚本 (Driver Code)
# ----------------------
def solve_2024():
    print("开始利用 Number 和 Call 类搜索 2024 ...")
    
    # 我们从复杂度 1 开始往上搜，直到找到为止
    # 比如 n=1, n=2, n=3 ...
    for n in range(1, 10): 
        print(f"正在搜索复杂度 n = {n} 的所有组合...")
        
        # 遍历这一层生成的所有对象
        for expr in smalls(n):
            if expr.value == 2024:
                return expr  # 找到了！直接返回这个对象

# ----------------------
# 5. 运行并打印
# ----------------------
result_obj = solve_2024()

print("\n" + "="*30)
print(f"找到结果对象类型: {type(result_obj)}")
print(f"最短表达式字符串: {result_obj}")  # 这里会自动调用 __str__
print(f"计算数值结果:     {result_obj.value}")
print("="*30)

开始利用 Number 和 Call 类搜索 2024 ...
正在搜索复杂度 n = 1 的所有组合...
正在搜索复杂度 n = 2 的所有组合...
正在搜索复杂度 n = 3 的所有组合...
正在搜索复杂度 n = 4 的所有组合...
正在搜索复杂度 n = 5 的所有组合...
正在搜索复杂度 n = 6 的所有组合...
正在搜索复杂度 n = 7 的所有组合...

找到结果对象类型: <class '__main__.Call'>
最短表达式字符串: g(h(g(5), g(g(f(f(5))))))
计算数值结果:     2024
